## 5. Checking the model on a cat vs dog dataset
From Kaggle we obtained 200 images of cats or dogs. Here we used the same ResNet50 pre-trained model to identify cats vs dogs. The results show improvement compared to the x-ray data. It suggests that the cavity data could be too difficult to identify, or ResNet50 is not working so well with x-ray data. 

In [15]:
import cv2
import numpy as np
import imageio
import os
import pandas as pd

In [16]:
from keras.applications.resnet50 import ResNet50, preprocess_input

HEIGHT = 300
WIDTH = 300

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH,3))


In [17]:
from keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = "test"
HEIGHT = 300
WIDTH = 300
BATCH_SIZE = 8

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True,
      validation_split=0.2
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE,
                                                    subset='training')


validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, # same directory as training data
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    subset='validation') 


Found 162 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [18]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

class_list = ["cat", "dog"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))

In [21]:
from keras.optimizers import SGD, Adam

NUM_EPOCHS = 10
BATCH_SIZE = 8
num_train_images = 202

adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

#filepath="./checkpoints/" + "ResNet50" + "_model_weights.h5"
#checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
#callbacks_list = [checkpoint]

#history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       #steps_per_epoch=num_train_images // BATCH_SIZE, 
                                    #   shuffle=True, callbacks=callbacks_list)
        
        
#history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       #steps_per_epoch=num_train_images // BATCH_SIZE, 
                                       #shuffle=True)


history= finetune_model.fit_generator(train_generator,epochs=NUM_EPOCHS, workers=8, 
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    shuffle=True)    
    
    

Epoch 1/10
20/20 [==============================] - 217s 11s/step - loss: 0.5174 - acc: 0.8685 - val_loss: 0.3915 - val_acc: 0.8250
Epoch 2/10
20/20 [==============================] - 211s 11s/step - loss: 0.6352 - acc: 0.8193 - val_loss: 0.7197 - val_acc: 0.7750
Epoch 3/10
20/20 [==============================] - 213s 11s/step - loss: 0.4108 - acc: 0.8998 - val_loss: 0.2790 - val_acc: 0.9250
Epoch 4/10
20/20 [==============================] - 222s 11s/step - loss: 0.5725 - acc: 0.8625 - val_loss: 1.1723 - val_acc: 0.7000
Epoch 5/10
20/20 [==============================] - 216s 11s/step - loss: 0.5297 - acc: 0.8316 - val_loss: 0.3670 - val_acc: 0.9250
Epoch 6/10
20/20 [==============================] - 204s 10s/step - loss: 0.5686 - acc: 0.8748 - val_loss: 0.4886 - val_acc: 0.8750
Epoch 7/10
20/20 [==============================] - 207s 10s/step - loss: 0.3501 - acc: 0.8820 - val_loss: 0.3616 - val_acc: 0.9000
Epoch 8/10
20/20 [==============================] - 232s 12s/step - loss: 0.

In [22]:


# Plot the training and validation loss + accuracy
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    # plt.figure()
    # plt.plot(epochs, loss, 'r.')
    # plt.plot(epochs, val_loss, 'r-')
    # plt.title('Training and validation loss')
    plt.show()

    plt.savefig('acc_vs_epochs.png')
    

In [23]:
plot_training(history)

NameError: name 'plt' is not defined